<a href="https://colab.research.google.com/github/wdittaya/MLWorkshop/blob/main/CUVIP_SupervisedLearningWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset
This dataset contains voltage, current, power, energy, and weather data from low-voltage substations and domestic premises with high uptake of solar photovoltaic (PV) embedded generation.

Key stats about the dataset:

- 20 substations and 10 domestic premises
- Collected over 480 days - 27 July 2013 to 19 November 2014
- 10 minute intervals over all time recorded, 1 minute intervals in summer 2014
- 10-minute measurements prior to 10 June 2014, aggregated to hourly minima and maxima

Ref: https://data.london.gov.uk/dataset/photovoltaic--pv--solar-panel-energy-generation-data

# โจทย์

1. ทำนายกำลังการผลิตไฟฟ้าใน 1 ชั่วโมงข้างหน้าจากข้อมูลสภาวะอากาศย้อนหลัง 3 ชั่วโมง
2. ทำนายปริมาณการซื้อไฟฟ้าล่วงหน้า 1 ชั่วโมง จากข้อมูลการใช้ไฟฟ้าและการผลิตไฟฟ้าย้อนหลัง 3 ชั่วโมง

# Download ข้อมูล

ข้อมูลการผลิตไฟฟ้า

In [ ]:
!wget https://data.london.gov.uk/download/photovoltaic--pv--solar-panel-energy-generation-data/81fb6b31-f6b2-4e12-b054-090319faec7b/PV%20Data.zip
!unzip 'PV Data.zip'
!unzip 'PV Data - csv files only.zip'

ข้อมูลสภาพอากาศ

In [ ]:
!wget https://data.london.gov.uk/download/photovoltaic--pv--solar-panel-energy-generation-data/b4a7e790-8cb8-451c-b828-c4c5d8445705/Weather%20Data%202014-11-30.xlsx

เลือกใช้ข้อมูลการผลิตไฟฟ้าจากบ้านลูกค้า (customer endpoint) ราย 10 นาที

In [ ]:
import pandas as pd

customer_data_df = pd.read_csv('/content/2014-11-28 Cleansed and Processed/EXPORT TenMinData/EXPORT TenMinData - Customer Endpoints.csv')

In [ ]:
customer_data_df.head()

In [ ]:
print(customer_data_df.iloc[0])

Fields ที่จะใช้
- `Substation` สถานีย่อย
- `d_y` ปี
- `d_m` เดือน
- `d_d` วันที่
- `d_w` วันของสัปดาห์
- `t_h` ชั่วโมง
- `t_m` นาที
- `P_GEN` ปริมาณที่ผลิตเพื่อใช้ (kWatt)
- `P_IMPORT` ปริมาณไฟฟ้าที่ต้องซื้อ (kWatt)
    - ค่าเป็นบวกกรณีซื้อไฟฟ้า (ผลิตไม่พอใช้)
    - ค่าเป็นลบกรณีขายไฟฟ้า (ผลิตเกินที่ใช้)

In [ ]:
customer_data_df.columns

In [ ]:
customer_data_df['Substation'].unique()

# Preprocessing
- PV Data
    - เลือกข้อมูลที่ต้องการ
    - พิจารณาความจำเป็นในการ normalize data
- Weather data
- ปรับข้อมูลให้เป็นคู่ข้อมูลเข้า-ออก

## PV Data

### เลือกข้อมูลที่ต้องการ

In [ ]:
data_df = customer_data_df[['Substation', 'd_y', 'd_m', 'd_d', 'd_w', 't_h', 't_m', 'P_GEN', 'P_IMPORT']]
data_df.head()

### พิจารณาความจำเป็นในการ normalize ข้อมูล

ทดลอง plot ข้อมูล `P_GEN` ของวันที่ 1 กรกฎาคม 2014 แยกแต่ละ substation

In [ ]:
import matplotlib.pyplot as plt

# Filter data for August 1, 2014
jul1_2014_data = data_df[(data_df['d_y'] == 2014) & (data_df['d_m'] == 7) & (data_df['d_d'] == 1)]

# Aggregate data to hourly level
hourly_data = jul1_2014_data.groupby(['Substation', 't_h'])['P_GEN'].sum().reset_index()

# Plot P_GEN for each substation
plt.figure(figsize=(12, 6))
for substation in hourly_data['Substation'].unique():
  substation_data = hourly_data[hourly_data['Substation'] == substation]
  plt.plot(substation_data['t_h'], substation_data['P_GEN'], label=substation)

plt.xlabel('Hour')
plt.ylabel('P_GEN (kW)')
plt.title('Hourly P_GEN on July 1, 2014 by Substation')
plt.legend()
plt.grid(True)
plt.show()

ทดลอง plot ข้อมูลเฉลี่ยรายชั่วโมงของทั้ง dataset แยกตาม substation

In [ ]:
# Group data by substation and hour, and calculate the average P_GEN for each hour
hourly_avg_pgen = data_df.groupby(['Substation', 'd_y', 'd_m', 'd_d', 't_h'])['P_GEN'].sum().reset_index().groupby(['Substation', 't_h'])['P_GEN'].mean().reset_index()

# Plot the average hourly P_GEN for each substation
plt.figure(figsize=(15, 8))
for substation in hourly_avg_pgen['Substation'].unique():
  substation_data = hourly_avg_pgen[hourly_avg_pgen['Substation'] == substation]
  plt.plot(substation_data['t_h'], substation_data['P_GEN'], label=substation)

plt.xlabel('Hour')
plt.ylabel('Average P_GEN (kW)')
plt.title('Average Hourly P_GEN by Substation')
plt.legend()
plt.grid(True)
plt.show()

จากกราฟ จะเห็นว่าปริมาณไฟฟ้าที่ผลิตได้ของแต่ละ substation ไม่เท่ากัน ไม่น่าจะใช้ข้อมูลใน substation หนึ่งไปทำนายปริมาณการผลิตในอีก substation หนึ่งได้โดยตรง จึงไม่น่าจะเอาข้อมูลทุก substation มารวมกันได้ และน่าจะต้องเลือกทำนายเป็น substation ไป

### ทำข้อมูลรายชั่วโมง

sum ข้อมูลในชั่วโมงเดียวกันเข้าด้วยกัน

In [ ]:
# prompt: Aggregate data_df into hourly data

# Aggregate data to hourly level
hourly_data_df = data_df.groupby(['Substation', 'd_y', 'd_m', 'd_d', 't_h'])[['P_GEN', 'P_IMPORT']].sum().reset_index()

hourly_data_df.head()

## Weather data

In [ ]:
# Read weather data
weather_df = pd.read_excel('/content/Weather Data 2014-11-30.xlsx')

In [ ]:
weather_df.head()

In [ ]:
print(weather_df.iloc[0])

In [ ]:
weather_df['Site'].unique()

In [ ]:
data_df['Substation'].unique()

จะเห็นว่า มีเฉพาะ
- YMCA
- Maple Drive East
- Forest Road

3 site/substation นี้เท่านั้น ที่มีข้อมูลสภาพอากาศ

In [ ]:
# Find intersection of Site and Substation
site_selection = set(weather_df['Site'].unique()).intersection(set(data_df['Substation'].unique()))
print(site_selection)

Fields ที่จะใช้
- `Site` ชื่อสถานที่ติดตั้งแผง
- `Date` วันที่ เดือน ปี (ไม่รวมเวลา)
- `Time` เวลา (ชั่วโมง นาที)
- `TempOut` อุณหภูมิภายนอกอาคาร
- `OutHum` ความชื้นภายนอกอาคาร
- `DewPt` อุณหภูมิจุดน้ำค้าง
- `WindSpeed` ความเร็วลม
- `WindDir` ทิศทางลม
- `WindRun` ปริมาณลม
- `WindChill` ความเย็นจากลม
- `Bar` ความกดอากาศ
- `Rain` ปริมาณน้ำฝน
- `SolarRad` ความเข้มแสงอาทิตย์

In [ ]:
weather_df.columns

### เลือกข้อมูลที่ต้องการ

In [ ]:
features = ['Site', 'Date', 'Time', 'TempOut', 'OutHum', 'DewPt', 'WindSpeed', 'WindDir', 'WindRun', 'WindChill', 'Bar', 'Rain', 'SolarRad']
weather_data_df = weather_df[features]
weather_data_df.head()

เลือกเฉพาะ Site ที่มีข้อมูลสภาพอากาศ

In [ ]:
# Choose only site in site_selection
weather_data_df = weather_data_df[weather_data_df['Site'].isin(site_selection)]
weather_data_df.head()

### ตรวจสอบ missing values

In [ ]:
# prompt: Count missing values for each site

# Count missing values for each site in the weather data
weather_data_df.groupby('Site').apply(lambda x: x.isnull().sum())

ที่จริงแล้ว missing values ถูกแทนด้วย `---` ในข้อมูล

In [ ]:
# Count '---' in weahter data by site
weather_data_df.groupby('Site').apply(lambda x: x.isin(['---']).sum())

In [ ]:
# Count data entries for each site
weather_data_df.groupby('Site').size()

เติม missing value ด้วยข้อมูลก่อนหน้า (forward fill)

In [ ]:
# Forward fill missing values
weather_data_df = weather_data_df.replace('---', pd.NA)
weather_data_df = weather_data_df.fillna(method='ffill')

นับ Missing values อีกที

In [ ]:
# Count '---' in weahter data by site
weather_data_df.groupby('Site').apply(lambda x: x.isin(['---']).sum())

### แปลงข้อมูลวันที่ เวลา

In [ ]:
# prompt: Merge Date and Time from weather_data_df into Datetime
# Split Datetime to 'd_y', 'd_m', 'd_d', 'd_w', 't_h', 't_m'

# Merge Date and Time from weather_data_df into Datetime
weather_data_df['Datetime'] = pd.to_datetime(weather_data_df['Date'].astype(str) + ' ' + weather_data_df['Time'].astype(str))

# Split Datetime to 'd_y', 'd_m', 'd_d', 'd_w', 't_h', 't_m'
weather_data_df['d_y'] = weather_data_df['Datetime'].dt.year
weather_data_df['d_m'] = weather_data_df['Datetime'].dt.month
weather_data_df['d_d'] = weather_data_df['Datetime'].dt.day
weather_data_df['d_w'] = weather_data_df['Datetime'].dt.dayofweek
weather_data_df['t_h'] = weather_data_df['Datetime'].dt.hour
weather_data_df['t_m'] = weather_data_df['Datetime'].dt.minute

In [ ]:
weather_data_df.head()

### พิจารณาความจำเป็นในการ normalize ข้อมูล

ข้อมูลสภาพอากาศแบ่งออกเป็น
- Numeric weather feature
- Categorical weather feature

In [ ]:
numeric_features = ['TempOut', 'OutHum', 'DewPt', 'WindSpeed', 'WindRun', 'WindChill', 'Bar', 'Rain', 'SolarRad']
categorical_features = ['Site', 'WindDir']

#### Numeric features

ทดลอง plot ข้อมูลรายชั่วโมงแต่ละ weather data ของวันที่ 1 กรฎาคม 2024

In [ ]:
# prompt: plot ข้อมูลรายชั่วโมงแต่ละ numeric feature ของ weather data ในวันที่ 1 กรกฎาคม 2024 แยก Site

# Filter data for July 1, 2014
jul1_2014_weather_data = weather_data_df[(weather_data_df['d_y'] == 2014) & (weather_data_df['d_m'] == 7) & (weather_data_df['d_d'] == 1)]

# Group data by site and hour, and calculate the average of numeric features for each hour
hourly_weather = jul1_2014_weather_data.groupby(['Site', 't_h'])[numeric_features].sum().reset_index()

# Plot the average hourly weather features for each site
for feature in numeric_features:
  plt.figure(figsize=(15, 8))
  for site in hourly_weather['Site'].unique():
    site_data = hourly_weather[hourly_weather['Site'] == site]
    plt.plot(site_data['t_h'], site_data[feature], label=site)

  plt.xlabel('Hour')
  plt.ylabel(f'{feature}')
  plt.title(f'Hourly {feature} on July 1, 2014 by Site')
  plt.legend()
  plt.grid(True)
  plt.show()

ทดลอง plot ข้อมูลเฉลี่ยรายชั่วโมงของข้อมูลสภาพอากาศแต่ละค่า

In [ ]:
weather_data_df.groupby(['Site', 'd_y', 'd_m', 'd_d', 't_h'])[numeric_features].sum().reset_index().groupby(['Site', 't_h']).mean()[numeric_features]

In [ ]:
# prompt: Plot ข้อมูลเฉลี่ยรายชั่วโมงของ numeric weather data แยกตาม Site

# Group data by site and hour, and calculate the average of numeric features for each hour

hourly_avg_weather = weather_data_df.groupby(['Site', 'd_y', 'd_m', 'd_d', 't_h'])[numeric_features].sum().reset_index().groupby(['Site', 't_h']).mean()[numeric_features].reset_index()

# Plot the average hourly weather features for each site
for feature in numeric_features:
  plt.figure(figsize=(15, 8))
  for site in hourly_avg_weather['Site'].unique():
    site_data = hourly_avg_weather[hourly_avg_weather['Site'] == site]
    plt.plot(site_data['t_h'], site_data[feature], label=site)

  plt.xlabel('Hour')
  plt.ylabel(f'Average {feature}')
  plt.title(f'Average Hourly {feature} by Site')
  plt.legend()
  plt.grid(True)
  plt.show()

จะเห็นได้ว่า ลักษณะข้อมูลแต่ละพื้นที่นั้นต่างกัน มีค่าสูงสุด-ต่ำสุด ที่ไม่เหมือนกัน

นอกจากนี้ ข้อมูลสภาพอากาศแต่ละค่า ก็มี scale ที่ต่างกันค่อนข้างมาก

หากต้องการใช้ข้อมูลสภาพอากาศทั้งหมดมาใช้ในการทำนาย ต้องมีการ normalize ข้อมูลสภาพอากาศแต่ละค่าให้อยู่ใน scale เดียวกัน

#### Categorical features

- `Site` ใช้อ้างอิงเพื่อกรองข้อมูลเพื่อสร้าง model แยก ไม่จำเป็นต้องทำอะไรเพิ่ม
- `WindDir` ทิศทางลม


In [ ]:
weather_data_df['WindDir'].unique()

เมื่อพิจารณาแล้ว พบว่า ทิศทางลมเป็น category น่าจะใช้ one-hot vector แทนทิศทาง

In [ ]:
# Convert WindDir to OneHotEncoding
weather_data_df = pd.get_dummies(weather_data_df, columns=['WindDir'])

weather_data_df.head()

### ทำข้อมูลรายชั่วโมง

สร้าง list ของ boolean feature

In [ ]:
# prompt: Create list of boolean features from WindDir

# สร้าง list ของ boolean feature จาก WindDir
wind_dir_features = [col for col in weather_data_df.columns if col.startswith('WindDir_')]

print(wind_dir_features)

รวมข้อมูลในชั่วโมงเดียวกันเข้าด้วยกัน

In [ ]:
# Create hourly weather data for numeric features and boolean features
hourly_weather_df = weather_data_df.groupby(['Site', 'd_y', 'd_m', 'd_d', 't_h'])[numeric_features + wind_dir_features].sum().reset_index()

hourly_weather_df.head()

In [ ]:
hourly_weather_df.columns

# Create input-output dataset

In [ ]:
# prompt: Merge hourly_data_df with hourly_weather_df on Substation/Site, d_y, y_m, d_d, t_h
# Keep numeric_features and wind_dir_features

# Merge hourly_data_df with hourly_weather_df on Substation/Site, d_y, y_m, d_d, t_h
merged_df = pd.merge(hourly_data_df, hourly_weather_df, left_on=['Substation', 'd_y', 'd_m', 'd_d', 't_h'],
                     right_on=['Site', 'd_y', 'd_m', 'd_d', 't_h'], how='inner')

# Keep only numeric_features and wind_dir_features
selected_features = ['Site', 'd_y', 'd_m', 'd_d', 't_h'] + numeric_features + wind_dir_features + ['P_GEN', 'P_IMPORT']
merged_df = merged_df[selected_features]

merged_df.head()

In [ ]:
merged_df.info()

## Sample Site - YMCA

In [ ]:
# Choose data from YMCA Site
ymca_df = merged_df[merged_df['Site'] == 'YMCA']

In [ ]:
# prompt: Create datetime column from d_y, d_m, d_d, d_h

# Create datetime column from d_y, d_m, d_d, d_h
ymca_df['Datetime'] = pd.to_datetime(ymca_df[['d_y', 'd_m', 'd_d', 't_h']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H')

In [ ]:
ymca_df.head()

In [ ]:
# Drop Site, d_y from ymca_df
ymca_df = ymca_df.drop(['Site', 'd_y'], axis=1)

In [ ]:
ymca_df.columns

In [ ]:
# prompt: Set P_GEN and P_IMPORT to output
# The rest of columns are inputs
# Create time series input-output pair, using 3-hr history input data to predict outputs

# Assuming your dataframe is named 'ymca_df' and has 'P_GEN', 'P_IMPORT', and other columns as inputs.
import numpy as np

def create_timeseries_dataset(df, history_length=3):
  """
  Creates a time series input-output dataset using historical data.

  Args:
    df: Pandas DataFrame containing the data.
    history_length: Number of hours of historical data to use as input.

  Returns:
    A tuple of (inputs, outputs), where inputs is a numpy array of input data
    and outputs is a numpy array of corresponding output data.
  """

  inputs = []
  outputs = []

  for i in range(history_length, len(df)):
    input_data = df.iloc[i - history_length:i].values
    output_data = df[['P_GEN', 'P_IMPORT']].iloc[i].values
    inputs.append(input_data)
    outputs.append(output_data)

  return np.array(inputs), np.array(outputs)

# Sort ymca_df by datetime
ymca_df = ymca_df.sort_values('Datetime').drop('Datetime', axis=1)
inputs, outputs = create_timeseries_dataset(ymca_df)

print(inputs.shape)  # Shape of input data: (num_samples, history_length, num_features)
print(outputs.shape) # Shape of output data: (num_samples, 2)

In [ ]:
inputs[0]

In [ ]:
outputs[0]

# Setup Pipelines

## Simple linear regression

In [ ]:
# prompt: Setup sklearn pipeline
# - Normalize input values using MinMaxScaler
# - Create timeseries 5-fold cross validation
# - Train a regression model
# - Report error on each fold
# - Evaluate P_GEN and P_IMPORT prediction using RMSE

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming 'inputs' and 'outputs' are your prepared time series data

# Create a pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Normalize input data
    ('model', LinearRegression())  # Train a linear regression model
])

# Define the cross-validation strategy (TimeSeriesSplit)
tscv = TimeSeriesSplit(n_splits=5)

# Loop through the cross-validation folds
for train_index, test_index in tscv.split(inputs):
    X_train, X_test = inputs[train_index], inputs[test_index]
    y_train, y_test = outputs[train_index], outputs[test_index]

    # Fit the pipeline on the training data
    pipeline.fit(X_train.reshape(X_train.shape[0], -1), y_train)

    # Predict on the test data
    y_pred = pipeline.predict(X_test.reshape(X_test.shape[0], -1))

    # Calculate and report RMSE for P_GEN and P_IMPORT
    rmse_pgen = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
    rmse_pimport = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))

    print(f"Fold RMSE - P_GEN: {rmse_pgen:.4f}, P_IMPORT: {rmse_pimport:.4f}")


## เปรียบเทียบ LinearRegression, Ridge และ Lasso

In [ ]:
# prompt: Compare LinearRegression, Ridge, Lasso model using the same pipieline

# Define models to compare
models = [
    ("LinearRegression", LinearRegression()),
    ("Ridge", Ridge()),
    ("Lasso", Lasso()),
]

for model_name, model in models:
  print(f"\nEvaluating {model_name}:")

  # Create a pipeline with the current model
  pipeline = Pipeline([
      ('scaler', MinMaxScaler()),
      ('model', model)
  ])

  # RMSE
  rmse_pgen_list = []
  rmse_pimport_list = []

  # Loop through the cross-validation folds
  for train_index, test_index in tscv.split(inputs):
      X_train, X_test = inputs[train_index], inputs[test_index]
      y_train, y_test = outputs[train_index], outputs[test_index]

      # Fit the pipeline on the training data
      pipeline.fit(X_train.reshape(X_train.shape[0], -1), y_train)

      # Predict on the test data
      y_pred = pipeline.predict(X_test.reshape(X_test.shape[0], -1))

      # Calculate and report RMSE for P_GEN and P_IMPORT
      rmse_pgen = np.sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
      rmse_pimport = np.sqrt(mean_squared_error(y_test[:, 1], y_pred[:, 1]))

      print(f"Fold RMSE - P_GEN: {rmse_pgen:.4f}, P_IMPORT: {rmse_pimport:.4f}")

      rmse_pgen_list.append(rmse_pgen)
      rmse_pimport_list.append(rmse_pimport)

  # Find average RMSE
  avg_rmse_pgen = np.mean(rmse_pgen_list)
  avg_rmse_pimport = np.mean(rmse_pimport_list)

  print(f"\nAverage RMSE - P_GEN: {avg_rmse_pgen:.4f}, P_IMPORT: {avg_rmse_pimport:.4f}")

# แบบฝึกหัด

1. เปลี่ยน Site เป็น Forest Road
2. เปลี่ยน Site เป็น Maple Drive East

โมเดลใดแม่นที่สุดเมื่อใช้กับทั้งสาม site (YMCA, Forest Road, Maple East)

## Forest Road site

## Maple Drive East